#### How do actors affect revenue made and rating received?
We want to explore how actors affect the movie revenue and movie rating.
To do so, we want to perform regression and evaluate the coefficients each actor correspond to. 
We can perform linear regression look at the top 10 highest coefficients that correspond to certain actors.
What are the top10 actors with highest effect on revenue?

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

#load the statistical libraries
from statsmodels.stats import diagnostic
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data_folder = './data/'
movies = pd.read_table(data_folder + 'movie.metadata.tsv', names=['wiki_id','freebase_id','name','release_date','box_office_revenue','length','language','country','genre'])
movies.shape

In [34]:
# Removing movies from before 2000
movies = movies[movies['release_date'] >= '2000']
movies.shape

In [3]:
characters = pd.read_table(data_folder + 'character.metadata.tsv', names=['wiki_movie_id', 'freebase_movie_id', 'movie_release_date', 'character_name', 'actor_birth', 'actor_gender', 'actor_height', 'actor_ethnicity_freebase_id', 'actor_name', 'actor_age_release', 'freebase_character_actor_id', 'freebase_character_id', 'freebase_actor_id'])

In [4]:
movies

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [5]:
characters

,wiki_movie_id,freebase_movie_id,movie_release_date,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity_freebase_id,actor_name,actor_age_release,freebase_character_actor_id,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


In [6]:
# Merging movies with characters on wiki_id. 
movies_characters = pd.merge(left=movies, right=characters, left_on='wiki_id', right_on='wiki_movie_id')
movies_characters

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre,wiki_movie_id,...,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity_freebase_id,actor_name,actor_age_release,freebase_character_actor_id,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,NaN,NaN,NaN,NaN,NaN,Billy Morton,NaN,/m/0gchkcy,NaN,/m/0gc4lfm
450665,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,NaN,1982-01-28,NaN,NaN,NaN,Andrea Runge,19.0,/m/0gckh4f,NaN,/m/0gbx_rk
450666,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,NaN,NaN,F,NaN,NaN,Wendy Anderson,NaN,/m/0gcp8fv,NaN,/m/0gby01h
450667,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,NaN,NaN,NaN,NaN,NaN,Ariel Bastian,NaN,/m/0gdkb51,NaN,/m/0gdkb55


In [7]:
movies_characters.isna().sum()

wiki_id                             0
freebase_id                         0
name                                0
release_date                     9995
box_office_revenue             349641
length                          50470
language                            0
country                             0
genre                               0
wiki_movie_id                       0
freebase_movie_id                   0
movie_release_date               9995
character_name                 257875
actor_birth                    106145
actor_gender                    45609
actor_height                   295845
actor_ethnicity_freebase_id    344611
actor_name                       1228
actor_age_release              158113
freebase_character_actor_id         0
freebase_character_id          257865
freebase_actor_id                 815
dtype: int64

In [8]:
# Removing movies with NULL value in freebase_actor_id
movies_characters = movies_characters[movies_characters['freebase_actor_id'].notna()]
movies_characters.shape

(449854, 22)

In [10]:
# Checking number of unique movies
num_movies = movies_characters["wiki_id"].nunique()
num_movies

20246

In [11]:
# Checking number of unique actors in dataset
num_actors_unique = movies_characters["freebase_actor_id"].nunique()
num_actors_unique

71372

In [12]:
# Checking number of actors in dataset, counting actors multiple times if they play
# in multiple movies
count_actors = movies_characters["freebase_actor_id"].count()
count_actors

176255

In [13]:
# Number of actors on average per movie
count_actors / num_movies

8.705670255853008

In [14]:
# One Hot Encoding of Actors
movies_characters_dummy = pd.get_dummies(data=movies_characters, columns=['freebase_actor_id'])

In [15]:
movies_characters_dummy['wiki_id'].nunique()

20246

In [16]:
movies_characters_dummy['wiki_id'].count()

176255

In [18]:
movies_characters_dummy

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre,wiki_movie_id,...,freebase_actor_id_/m/0y58x,freebase_actor_id_/m/0y8r3,freebase_actor_id_/m/0ywqc,freebase_actor_id_/m/0yxl,freebase_actor_id_/m/0z05l,freebase_actor_id_/m/0z4s,freebase_actor_id_/m/0z9q0,freebase_actor_id_/m/0zcb7,freebase_actor_id_/m/0zcbl,freebase_actor_id_/m/0zjpz
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,0,0,0,0,0,0,0,0,0,0
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,0,0,0,0,0,0,0,0,0,0
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,0,0,0,0,0,0,0,0,0,0
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,0,0,0,0,0,0,0,0,0,0
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,0,0,0,0,0,0,0,0,0,0
450665,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,0,0,0,0,0,0,0,0,0,0
450666,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,0,0,0,0,0,0,0,0,0,0
450667,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",12476867,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Only include wiki_id and one hot encoding of actors in dataframe
dummy_actor_columns = movies_characters_dummy.loc[ : , ~movies_characters_dummy.columns.isin(['freebase_id','name','release_date','box_office_revenue','length','language','country','genre', 'wiki_movie_id', 'freebase_movie_id', 'movie_release_date', 'character_name', 'actor_birth', 'actor_gender', 'actor_height', 'actor_ethnicity_freebase_id', 'actor_name', 'actor_age_release', 'freebase_character_actor_id', 'freebase_character_id', 'freebase_actor_id'])]

In [20]:
# Only include actor columns that correspond to actors that have played in more than xx movies
# Current threshold is 20. 
# TODO: Decide on threshold
dummy_actor_columns = dummy_actor_columns.loc[:, dummy_actor_columns.sum(axis=0) > 20]

In [22]:
# Grouping movies such that every movie correspond to only one row in the dataframe
dummy_actor_columns = dummy_actor_columns.groupby('wiki_id').agg('sum')

In [29]:
dummy_actor_columns

,freebase_actor_id_/m/011zd3,freebase_actor_id_/m/0126rp,freebase_actor_id_/m/012d40,freebase_actor_id_/m/012g92,freebase_actor_id_/m/012q4n,freebase_actor_id_/m/012x2b,freebase_actor_id_/m/014g22,freebase_actor_id_/m/014g_s,freebase_actor_id_/m/014gf8,freebase_actor_id_/m/014v6f,...,freebase_actor_id_/m/0pz91,freebase_actor_id_/m/0q9kd,freebase_actor_id_/m/0svqs,freebase_actor_id_/m/0sw6g,freebase_actor_id_/m/0sw6y,freebase_actor_id_/m/0sz28,freebase_actor_id_/m/0tj9,freebase_actor_id_/m/0ywqc,freebase_actor_id_/m/0z4s,freebase_actor_id_/m/0zcbl
wiki_id,,,,,,,,,,,,,,,,,,,,,
5313,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19715,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20786,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22216,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37322106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37373877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
movies_characters = movies_characters[['wiki_id', 'release_date', 'freebase_id','name','release_date','box_office_revenue','length','language','country','genre']].drop_duplicates()

In [56]:
# Concatenating dummy actor columns with movies
movies_binary_actors = pd.merge(movies_characters, dummy_actor_columns, left_on='wiki_id', right_index=True)

movies_binary_actors

,wiki_id,release_date,freebase_id,name,release_date,box_office_revenue,length,language,country,genre,...,freebase_actor_id_/m/0pz91,freebase_actor_id_/m/0q9kd,freebase_actor_id_/m/0svqs,freebase_actor_id_/m/0sw6g,freebase_actor_id_/m/0sw6y,freebase_actor_id_/m/0sz28,freebase_actor_id_/m/0tj9,freebase_actor_id_/m/0ywqc,freebase_actor_id_/m/0z4s,freebase_actor_id_/m/0zcbl
0,975900,2001-08-24,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",...,0,0,0,0,0,0,0,0,0,0
17,3196793,2000-02-16,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",...,0,0,0,0,0,0,0,0,0,0
42,18998739,2002,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...",...,0,0,0,0,0,0,0,0,0,0
122,21926710,2009,/m/05p45cv,White on Rice,2009,NaN,82.0,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/02l7c8"": ""...",...,0,0,0,0,0,0,0,0,0,0
149,20604092,2007-08-17,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,"{""/m/07c9s"": ""Tamil Language""}","{""/m/03rk0"": ""India""}","{""/m/02l7c8"": ""Romance Film""}",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450560,1918494,2000-08-26,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana...",...,0,0,0,0,0,0,0,0,0,0
450601,3868432,2003-08-14,/m/0b44p5,Into the Mirror,2003-08-14,NaN,113.0,"{""/m/02hwhyv"": ""Korean Language""}","{""/m/06qd3"": ""South Korea""}","{""/m/03npn"": ""Horror""}",...,0,0,0,0,0,0,0,0,0,0
450620,26482675,2010-03-05,/m/0bbwngb,Eşrefpaşalılar,2010-03-05,1847671.0,NaN,{},{},"{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...",...,0,0,0,0,0,0,0,0,0,0
450634,32468537,2007-10-18,/m/0crwd9y,Shadow Boxing 2,2007-10-18,NaN,132.0,"{""/m/06b_j"": ""Russian Language"", ""/m/02h40lc"":...","{""/m/06bnz"": ""Russia""}","{""/m/01z02hx"": ""Sports"", ""/m/0lsxr"": ""Crime Fi...",...,0,0,0,0,0,0,0,0,0,0


In [78]:
# Removing slashes in column names to avoid error in regression. 
movies_binary_actors.columns = movies_binary_actors.columns.str.replace('/', '')
dummy_actor_columns.columns = dummy_actor_columns.columns.str.replace('/', '')

In [ ]:
# One Hot Encoding of Genres
import json

for dict_ in movies_characters['genre']:
    # using json.loads()
    # convert dictionary string to dictionary
    dict_ = json.loads(dict_)
    
for key, val in dict_.items():
    print(key, val)

In [ ]:
# Making a dataframe with genre as columns
# Movie has 1 or 0 in genre columns depending on whether the movie includes the corresponding genre
binary_genre_df = pd.get_dummies(movies_characters['genre'])
binary_genre_df.sum()

In [103]:
# Constructing formula used for regression
formula = 'box_office_revenue ~ ' # length+', add continous variables later
for col in dummy_actor_columns.columns:
    formula += 'C(' + col + ')+'
    
formula = formula[:-1]

In [72]:
# Standardization of continuous predictors
movies_binary_actors['length'] = (movies_binary_actors['length'] - movies_binary_actors['length'].mean())/movies_binary_actors['length'].std()

In [104]:
# Linear regression
mod = smf.ols(formula=formula, data=movies_binary_actors)
res = mod.fit()
res_summary = res.summary()

In [106]:
res_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     box_office_revenue   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.460
Method:                 Least Squares   F-statistic:                     5.191
Date:                Mon, 14 Nov 2022   Prob (F-statistic):          1.13e-221
Time:                        09:47:27   Log-Likelihood:                -70882.
No. Observations:                3585   AIC:                         1.432e+05
Df Residuals:                    2856   BIC:                         1.477e+05
Df Model:                         728                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                            2.252e+07   2.72e+06      8.270      0.000    1.72e+07    2.79e+07
C(freebase_actor_id_m011zd3)[T.1]    3.765e+07   3.01e+07      1.250      0.211   -2.14e+07    9.67e+07
C(freebase_actor_id_m0126rp)[T.1]   -2.046e+07   3.68e+07     -0.556      0.578   -9.26e+07    5.17e+07
C(freebase_actor_id_m0126rp)[T.2]   -4.479e+17   2.07e+18     -0.217      0.829    -4.5e+18    3.61e+18
C(freebase_actor_id_m012d40)[T.1]    1.143e+08   3.72e+07      3.070      0.002    4.13e+07    1.87e+08
C(freebase_actor_id_m012g92)[T.1]   -1.137e+07   5.51e+07     -0.207      0.836   -1.19e+08    9.66e+07
C(freebase_actor_id_m012q4n)[T.1]    3.038e+07   2.99e+07      1.017      0.309   -2.82e+07     8.9e+07
C(freebase_actor_id_m012x2b)[T.1]    1.362e+07   3.56e+07      0.383      0.702   -5.61e+07    8.34e+07
C(freebase_actor_id_m014g22)[T.1]    3.855e+07   3.06e+07      1.260      0.208   -2.15e+07    9.86e+07
C(freebase_actor_id_m014g_s)[T.1]    8.896e+07   3.24e+07      2.749      0.006    2.55e+07    1.52e+08
C(freebase_actor_id_m014g_s)[T.2]    8.992e+07   1.05e+08      0.859      0.391   -1.15e+08    2.95e+08
C(freebase_actor_id_m014gf8)[T.1]    1.011e+08   3.32e+07      3.046      0.002     3.6e+07    1.66e+08
C(freebase_actor_id_m014v6f)[T.1]    1.473e+07    3.7e+07      0.398      0.691   -5.78e+07    8.73e+07
C(freebase_actor_id_m014x77)[T.1]   -5.582e+06   3.05e+07     -0.183      0.855   -6.55e+07    5.43e+07
C(freebase_actor_id_m014x77)[T.2]    1.141e+08   8.72e+07      1.308      0.191   -5.69e+07    2.85e+08
C(freebase_actor_id_m014zcr)[T.1]    3.098e+07   2.75e+07      1.128      0.259   -2.29e+07    8.48e+07
C(freebase_actor_id_m01515w)[T.1]   -2.882e+07   3.35e+07     -0.861      0.389   -9.44e+07    3.68e+07
C(freebase_actor_id_m01515w)[T.2]   -2.344e+05   7.24e+07     -0.003      0.997   -1.42e+08    1.42e+08
C(freebase_actor_id_m0151ns)[T.1]    4.948e+07   3.42e+07      1.448      0.148   -1.75e+07    1.17e+08
C(freebase_actor_id_m0151w_)[T.1]    5.053e+06   2.94e+07      0.172      0.863   -5.25e+07    6.26e+07
C(freebase_actor_id_m0151w_)[T.2]    3.791e+07    1.3e+08      0.292      0.770   -2.17e+08    2.92e+08
C(freebase_actor_id_m0154d7)[T.1]    -1.21e+08   7.88e+07     -1.537      0.124   -2.75e+08    3.34e+07
C(freebase_actor_id_m0154qm)[T.1]    4.904e+07   2.79e+07      1.761      0.078   -5.57e+06    1.04e+08
C(freebase_actor_id_m0159h6)[T.1]    1.497e+08   3.44e+07      4.353      0.000    8.22e+07    2.17e+08
C(freebase_actor_id_m015p37)[T.1]   -4.542e+07   3.86e+07     -1.177      0.239   -1.21e+08    3.03e+07
C(freebase_actor_id_m015rkw)[T.1]    -4.97e+06   2.93e+07     -0.170      0.865   -6.24e+07    5.25e+07
C(freebase_actor_id_m015t56)[T.1]   -5.924e+07   3.95e+07     -1.50

In [105]:
# Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
res_as_html = res_summary.tables[1].as_html()
summary_df = pd.read_html(res_as_html, header=0, index_col=0)[0]
summary_df

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.252000e+07,2.720000e+06,8.270,0.000,1.720000e+07,2.790000e+07
C(freebase_actor_id_m011zd3)[T.1],3.765000e+07,3.010000e+07,1.250,0.211,-2.140000e+07,9.670000e+07
C(freebase_actor_id_m0126rp)[T.1],-2.046000e+07,3.680000e+07,-0.556,0.578,-9.260000e+07,5.170000e+07
C(freebase_actor_id_m0126rp)[T.2],-4.479000e+17,2.070000e+18,-0.217,0.829,-4.500000e+18,3.610000e+18
C(freebase_actor_id_m012d40)[T.1],1.143000e+08,3.720000e+07,3.070,0.002,4.130000e+07,1.870000e+08
...,...,...,...,...,...,...
C(freebase_actor_id_m0tj9)[T.1],1.286000e+07,1.070000e+08,0.120,0.905,-1.980000e+08,2.230000e+08
C(freebase_actor_id_m0tj9)[T.2],0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00
C(freebase_actor_id_m0ywqc)[T.1],2.087000e+07,2.820000e+07,0.740,0.459,-3.440000e+07,7.620000e+07
C(freebase_actor_id_m0z4s)[T.1],7.943000e+07,3.050000e+07,2.605,0.009,1.970000e+07,1.390000e+08


In [96]:
# Only including actors with p-value < 0.05.
# Statistically significant actors
summary_df = summary_df[summary_df['P>|t|'] < 0.05]

In [97]:
# Sorting summary according to coefficients
summary_df.sort_values(by='coef', ascending=False)

,coef,std err,t,P>|t|,[0.025,0.975]
C(freebase_actor_id_m039v4h)[T.2],642500000.0,83000000.0,7.742,0.000,4.800000e+08,805000000.0
C(freebase_actor_id_m01wy5m)[T.2],637500000.0,159000000.0,4.019,0.000,3.270000e+08,948000000.0
C(freebase_actor_id_m03j367r)[T.1],607300000.0,117000000.0,5.173,0.000,3.770000e+08,837000000.0
C(freebase_actor_id_m0f5xn)[T.2],548600000.0,116000000.0,4.721,0.000,3.210000e+08,776000000.0
C(freebase_actor_id_m016z2j)[T.2],501900000.0,128000000.0,3.934,0.000,2.520000e+08,752000000.0
...,...,...,...,...,...,...
C(freebase_actor_id_m02ck7w)[T.1],-102800000.0,40800000.0,-2.523,0.012,-1.830000e+08,-22900000.0
C(freebase_actor_id_m0pgjm)[T.1],-118100000.0,44500000.0,-2.654,0.008,-2.050000e+08,-30800000.0
C(freebase_actor_id_m0582cf)[T.3],-297300000.0,140000000.0,-2.130,0.033,-5.710000e+08,-23600000.0
C(freebase_actor_id_m03q91d)[T.2],-448700000.0,147000000.0,-3.055,0.002,-7.370000e+08,-161000000.0
